In [1]:
import sqlite3
import os
import pandas as pd

connexion = sqlite3.connect("/home/juliendesmedt/Documents/becode/data_viz_SQL/input_data/bce.db")
cursor = connexion.cursor()

In [2]:
# query_all = """Select EnterpriseNumber
# ,	Status
# ,	StartDate
# ,	JuridicalSituation.Description as JuridicalSituation
# ,	TypeOfEnterprise.Description as TypeOfEnterprise
# ,	JuridicalForm.Description as JuridicalForm

# FROM enterprise

# LEFT JOIN (Select code, Description
# from code
# where code.Category = "TypeOfEnterprise" AND code."Language"="FR") as TypeOfEnterprise
# ON enterprise.TypeOfEnterprise=TypeOfEnterprise.code

# LEFT JOIN (Select code, Description
# from code
# where code.Category = "JuridicalForm" AND code."Language"="FR") as JuridicalForm
# ON enterprise.JuridicalForm=JuridicalForm.code

# LEFT JOIN (Select code, Description
# from code
# where code.Category = "JuridicalSituation" AND code."Language"="FR") as JuridicalSituation
# ON enterprise.JuridicalSituation=JuridicalSituation.code"""

# query_enterprise = """
# Select *
# FROM enterprise
# """
# query_JuridicalSituation ="""
# Select code, Description as JuridicalSituation
# from code
# where code.Category = "JuridicalSituation" AND code."Language"="FR"
# """

# query_JuridicalForm ="""
# Select code, Description as JuridicalForm
# from code
# where code.Category = "JuridicalForm" AND code."Language"="FR"
# """
# query_TypeOfEnterprise ="""
# Select code, Description as TypeOfEnterprise
# from code
# where code.Category = "TypeOfEnterprise" AND code."Language"="FR"
# """


for i in [ 'JuridicalSituation', 'JuridicalForm', 'TypeOfEnterprise' ]:
    cursor = connexion.cursor()
    cursor = cursor.execute( f"""
Select  Code, Description as "b_{i}"
from code
where code.Category = "{i}" AND code."Language"="FR";
""")
    cursor = cursor.fetchall()
    col = [description[0] for description in cursor.description]
    f"{i}" = pd.DataFrame.from_records(data=cursor, columns=col)

In [2]:
cursor.execute("""
Select *
FROM enterprise
""")
enterprise = cursor.fetchall()
col = [description[0] for description in cursor.description]

enterprise = pd.DataFrame.from_records(data=enterprise, columns=col)

cursor = cursor.execute( """
Select Code, Description
from code
where code.Category = "JuridicalSituation" AND code."Language"="FR";
""")
JuridicalSituation = cursor.fetchall()
col = [description[0] for description in cursor.description]
JuridicalSituation = pd.DataFrame.from_records(data=JuridicalSituation, columns=col)

cursor = cursor.execute( """
Select  Code, Description
from code
where code.Category = "JuridicalForm" AND code."Language"="FR";
""")
JuridicalForm = cursor.fetchall()
col = [description[0] for description in cursor.description]
JuridicalForm = pd.DataFrame.from_records(data=JuridicalForm, columns=col)

#TypeOfEnterprise
cursor = cursor.execute( """
Select Code, Description
from code
where code.Category = "TypeOfEnterprise" AND code."Language"="FR";
""")
TypeOfEnterprise = cursor.fetchall()
col = [description[0] for description in cursor.description]
TypeOfEnterprise = pd.DataFrame.from_records(data=TypeOfEnterprise, columns=col)

In [ ]:
# for i in [ JuridicalSituation, JuridicalForm, TypeOfEnterprise ]:
#     enterprise = enterprise.set_index(i).join(i.set_index('Code'), how='left')\
#         .rename(columns={'Description' : i})

In [3]:
enterprise = enterprise.set_index('JuridicalSituation').join(JuridicalSituation.set_index('Code'), how='left')\
    .rename(columns={'Description' : 'JuridicalSituation'})

enterprise = enterprise.set_index('JuridicalForm').join(JuridicalForm.set_index('Code'), how='left')\
    .rename(columns={'Description' : 'JuridicalForm'})

enterprise = enterprise.set_index('TypeOfEnterprise').join(TypeOfEnterprise.set_index('Code'), how='left')\
    .rename(columns={'Description' : 'TypeOfEnterprise'})    

In [4]:
enterprise

,EnterpriseNumber,Status,StartDate,JuridicalSituation,JuridicalForm,TypeOfEnterprise
1,0500.017.677,AC,2016-01-01 00:00:00,Situation normale,NaN,Personne physique
1,0500.044.797,AC,1800-01-01 00:00:00,Situation normale,NaN,Personne physique
1,0500.081.916,AC,1971-01-01 00:00:00,Situation normale,NaN,Personne physique
1,0500.083.993,AC,1965-01-01 00:00:00,Situation normale,NaN,Personne physique
1,0500.085.082,AC,2021-04-01 00:00:00,Situation normale,NaN,Personne physique
...,...,...,...,...,...,...
2,0859.371.993,AC,1986-07-01 00:00:00,Situation normale,Forme légale inconnue (ONSS),Personne morale
2,0859.372.092,AC,2000-07-01 00:00:00,Situation normale,Forme légale inconnue (ONSS),Personne morale
2,0859.380.804,AC,1982-07-01 00:00:00,Situation normale,Forme légale inconnue (ONSS),Personne morale
2,0850.392.169,AC,1970-01-01 00:00:00,Dissolution de plein droit par arrivée du terme,Forme légale inconnue (ONSS),Personne morale


In [18]:
group_data = (enterprise.groupby('JuridicalSituation')['EnterpriseNumber']
                .count()
                .reset_index(name='nmb_JurSit') )

In [27]:
group_data['%']=group_data['nmb_JurSit']/enterprise['EnterpriseNumber'].count()*100
group_data

#remove the other en create a separate frame 
if group_data['JuridicalSituation'] is not ['Dissolution anticipée - Liquidation (dissolution volontaire)','Ouverture de faillite','Situation normale']:
    group_data['cleaned_JuridicalSituation']='other'
    else: group_data['cleaned_JuridicalSituation']


,JuridicalSituation,nmb_JurSit,%
0,Arrêt des activités en Belgique (entité étrang...,3,0.000161
1,Clôture de faillite,1,0.000054
2,Clôture de faillite avec excusabilité,3,0.000161
3,Concordat après faillite,18,0.000966
4,Concordat avant faillite,424,0.022755
5,Dissolution anticipée - Liquidation (dissoluti...,25827,1.386095
6,Dissolution de plein droit par arrivée du terme,649,0.034831
7,Dissolution judiciaire ou nullité,3233,0.173510
8,Fin du sursis,43,0.002308
9,Ouverture de faillite,74732,4.010751


In [25]:
list(group_data['JuridicalSituation'].T)

['Arrêt des activités en Belgique (entité étrangère)',
 'Clôture de faillite',
 'Clôture de faillite avec excusabilité',
 'Concordat après faillite',
 'Concordat avant faillite',
 'Dissolution anticipée - Liquidation (dissolution volontaire)',
 'Dissolution de plein droit par arrivée du terme',
 'Dissolution judiciaire ou nullité',
 'Fin du sursis',
 'Ouverture de faillite',
 'Ouverture de faillite avec excusabilité',
 'Ouverture de faillite avec inexcusabilité',
 'Réouverture de liquidation',
 'Réunion des parts en une seule main',
 'Révocation du sursis',
 'Situation normale',
 'Sursis (réorganisation judiciaire)',
 'Sursis définitif',
 'Sursis provisoire']